In [1]:
module O

using Distributions
using LinearAlgebra
using Random
using Printf

function pvalue_N(X)
    n = length(X)
    z = sum(X) / √n
    ccdf(Chisq(1), z^2)
end

function pvalue_F(X)
    n = length(X)
    χ² = dot(X, X)
    ccdf(Chisq(n), χ²)
end

struct SimPval{P,T}
    pval_N::P
    pval_F::P
    α::T
end

function Base.show(io::IO, ::MIME"text/plain", x::SimPval)
    (; pval_N, pval_F, α) = x
    Nrej = pval_N .< α
    Nacc = .!Nrej
    Frej = pval_F .< α
    Facc = .!Frej
    print(io, @sprintf("alpha error rate for N: %6.3f\n", mean(Nrej)))
    print(io, @sprintf("alpha error rate for F: %6.3f\n", mean(Frej)))
    print(io, "\n")
    print(io, @sprintf("          %8s  %8s\n", "reject F", "accept F"))
    print(io, @sprintf("reject N %8.3f  %8.3f\n", mean(Nrej .& Frej), mean(Nrej .& Facc)))
    print(io, @sprintf("accept N %8.3f  %8.3f\n", mean(Nacc .& Frej), mean(Nacc .& Facc)))
end

function sim_pval(; n=20, niters=10^6, α=0.05)
    pval_N = zeros(niters)
    pval_F = zeros(niters)
    Xtmp = [randn(n) for _ in 1:Threads.nthreads()]
    Threads.@threads :static for i in 1:niters
        tid = Threads.threadid()
        X = rand!(Normal(), Xtmp[tid])
        pval_N[i] = pvalue_N(X)
        pval_F[i] = pvalue_F(X)
    end
    O.SimPval(pval_N, pval_F, α)
end

end

Main.O

モデル
$$
X_i \sim \mathrm{Normal}(\mu_i, 1) \quad \text{($i=1,2,\ldots,n$) かつ $X_i$達は独立}
$$

帰無仮説N:\
$$
\frac{\mu_1+\cdots+\mu_n}{n} = 0
$$

帰無仮説F:
$$
(\mu_1,\ldots,\mu_n)=(0,\ldots,0)
$$

In [2]:
O.sim_pval(; n=5, niters=10^7, α=0.05)

alpha error rate for N:  0.050
alpha error rate for F:  0.050

          reject F  accept F
reject N    0.014     0.036
accept N    0.036     0.913


In [3]:
O.sim_pval(; n=20, niters=10^7, α=0.05)

alpha error rate for N:  0.050
alpha error rate for F:  0.050

          reject F  accept F
reject N    0.007     0.043
accept N    0.043     0.907


In [4]:
O.sim_pval(; n=100, niters=10^7, α=0.05)

alpha error rate for N:  0.050
alpha error rate for F:  0.050

          reject F  accept F
reject N    0.004     0.046
accept N    0.046     0.904


In [5]:
O.sim_pval(; n=1000, niters=10^7, α=0.05)

alpha error rate for N:  0.050
alpha error rate for F:  0.050

          reject F  accept F
reject N    0.003     0.047
accept N    0.047     0.903
